## IMPORTING LIBRARIES

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
import numpy as np

## LOADING THE DATASET

In [ ]:
# Load the HIGGS dataset
df = pd.read_csv('/kaggle/input/higgs-uci-dataset/HIGGS.csv')

# Display the first few rows of the dataframe
print(df.head())

## STRATIFIED SAMPLING OF THE DATA

Stratified Sampling the data so that we get equal number of classes in output of the sampling.


In [ ]:
# Calculate the sample size (0.01% of the dataset)
sample_size = int(0.0001 * len(df))

# Assuming the target variable is the last column (0 for background, 1 for signal)
# Change 'target_column_name' to the actual name if different
target_column_name = '1.000000000000000000e+00'  # Replace with the actual column name if needed

# Create a stratified sample
stratified_sample, _ = train_test_split(df, train_size=sample_size, stratify=df[target_column_name], random_state=42)

# Display the stratified sample
print(stratified_sample)

## Display Basic Statistics

We begin by displaying basic statistics for the stratified sample using the `describe()` method. This provides an overview of the dataset's features.

Next, we calculate the number of rows and columns needed for subplots based on the number of features (excluding the target column). We set a fixed number of columns for the plots and compute the required rows.

### Feature Distribution Plots

We create histograms with Kernel Density Estimates (KDE) for each feature to visualize their distributions.

### Boxplots for Outliers

Boxplots are generated for each feature to identify potential outliers, providing insights into the spread and skewness of the data.


In [ ]:
# Display basic statistics
print("Basic Statistics:")
print(stratified_sample.describe())

# Calculate rows and columns for subplots based on number of features
num_features = len(stratified_sample.columns) - 1  # Exclude target column
cols = 5  # Choose number of columns
rows = math.ceil(num_features / cols)  # Calculate rows based on number of features

# Plot distributions of each feature
plt.figure(figsize=(20, 4 * rows))
for i, column in enumerate(stratified_sample.columns[1:], 1):  # Skip the target column
    plt.subplot(rows, cols, i)
    sns.histplot(stratified_sample[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Boxplots for outliers
plt.figure(figsize=(20, 4 * rows))
for i, column in enumerate(stratified_sample.columns[1:], 1):  # Skip the target column
    plt.subplot(rows, cols, i)
    sns.boxplot(x=stratified_sample[column])
    plt.title(f'Boxplot of {column}')
    plt.xlabel(column)

plt.tight_layout()
plt.show()


## NORMALISATION OF DATA

In [ ]:

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the features
X_normalized = scaler.fit_transform(stratified_sample)

# Convert back to DataFrame
X_normalized = pd.DataFrame(X_normalized, columns=stratified_sample.columns)

# Display the first few rows of the normalized data
print(X_normalized)
y=X_normalized['1.000000000000000000e+00']
# Drop a specific column (e.g., 'column_name') from X_normalized
X_normalized = X_normalized.drop('1.000000000000000000e+00', axis=1)


## FEATURE ENGINEERING

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Define the degree for polynomial features (e.g., 2 for quadratic)
degree = 2

# Initialize PolynomialFeatures with interaction_only=False to include polynomial terms
poly = PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False)

# Generate polynomial and interaction features for X
X_poly = poly.fit_transform(X_normalized)

# Display the shape and a few rows of the new feature matrix
print("Original feature matrix shape:", X_normalized.shape)
print("Polynomial feature matrix shape:", X_poly.shape)
print("Polynomial features (first 5 rows):\n", X_poly[:5])
X_normalized= pd.DataFrame(X_poly)


## RECURSIVE FEATURE ELIMINATION

In [ ]:

import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
 
# Split into train and test sets using X1 as features
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression(max_iter=1000)

# Create RFE model and select the top n features
rfe = RFE(estimator=model, n_features_to_select=20)  # Specify the number of features you want to select
rfe.fit(X_train, y_train)

# Get the selected features
selected_features_rfe = X1.columns[rfe.support_].tolist()
print("Selected features using RFE:", selected_features_rfe)

# Create a new DataFrame with the selected features
X_selected = X1[selected_features_rfe]

# Display the new DataFrame
print("DataFrame with selected features:")
print(X_selected.head())
X=X_selected
X.shape



## SPLITTING THE DATA FOR CROSS VALIDATION

In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Select the first two features for visualization (adjust if needed)
X_selected = X_train_scaled[:, :2]  # Select only two features for visualization

## LINEAR SVM IMPLEMENTATION

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline






# Check if we have exactly 2 features selected for visualization
if X_selected.shape[1] == 2:
    # Define and train the SVM model on the selected features
    svm_model_2d = SVC(kernel='linear', probability=True)
    svm_model_2d.fit(X_selected, y_train)

    # Create a mesh grid for plotting decision boundary
    x_min, x_max = X_selected[:, 0].min() - 1, X_selected[:, 0].max() + 1
    y_min, y_max = X_selected[:, 1].min() - 1, X_selected[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

    # Predict on the mesh grid
    Z = svm_model_2d.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the decision boundary
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.8, cmap='viridis')
    plt.scatter(X_selected[:, 0], X_selected[:, 1], c=y_train, edgecolors='k', marker='o', cmap='viridis')
    plt.title('SVM with Linear Kernel (Top 2 Features)')
    plt.xlabel('Feature 1 (standardized)')
    plt.ylabel('Feature 2 (standardized)')
    plt.show()
else:
    print("The dataset must have exactly 2 features for visualization.")

# Cross-validation on the full training set with all features
svm_model_full = make_pipeline(StandardScaler(), SVC(kernel='linear'))
cv_scores = cross_val_score(svm_model_full, X_train, y_train, cv=5)  # 5-fold cross-validation

# Print the cross-validation results
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", np.mean(cv_scores))

## ACCURACY METRICS

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, roc_curve
from sklearn.preprocessing import label_binarize

# Train the full SVM model with all features
svm_model_full.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = svm_model_full.predict(X_test_scaled)

# Calculate classification metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multiclass
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print classification metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# AUC score (only applicable for binary or one-vs-all multiclass)
# If binary classification:
y_pred_prob = svm_model_full.decision_function(X_test_scaled)
auc = roc_auc_score(y_test, y_pred_prob)
print("AUC Score:", auc)
    
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()




## STOCHASTIC GRADIENT DESCENT FOR SVM

In [ ]:

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report


# Create an SGDClassifier with SVM loss (Hinge Loss for SVM)
sgd_svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)

# Train the model using mini-batches
sgd_svm.fit(X_train_scaled, y_train)

# Make predictions
y_pred = sgd_svm.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)


## SVM WITH POLYNOMIAL KERNEL

In [ ]:

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import label_binarize

# Define the degrees to test
degrees = [2, 3, 4]

# Initialize a dictionary to store the results
results = {}

# Loop over each degree and evaluate the model
for degree in degrees:
    print(f"\n--- Polynomial Kernel with Degree {degree} ---")
    
    # Define and train the SVM model with a polynomial kernel
    svm_model_poly = SVC(kernel='poly', degree=degree, probability=True)
    svm_model_poly.fit(X_selected, y_train)  # Assuming X_selected contains only 2 features

    # Predict on the test set
    y_pred = svm_model_poly.predict(X_test_scaled[:, :2])  # Using the same 2 features for testing
    y_pred_prob = svm_model_poly.decision_function(X_test_scaled[:, :2])

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # For AUC, handle binary and multiclass cases separately
    if len(set(y)) == 2:
        auc = roc_auc_score(y_test, y_pred_prob)
    else:
        y_test_binarized = label_binarize(y_test, classes=np.unique(y))
        auc = roc_auc_score(y_test_binarized, y_pred_prob, average="weighted", multi_class="ovr")

    # Print results for the current degree
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"AUC Score: {auc:.2f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    # Store the results
    results[degree] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }

    # Plot the decision boundary
    x_min, x_max = X_selected[:, 0].min() - 1, X_selected[:, 0].max() + 1
    y_min, y_max = X_selected[:, 1].min() - 1, X_selected[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    
    Z = svm_model_poly.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    plt.scatter(X_selected[:, 0], X_selected[:, 1], c=y_train, edgecolor='k', cmap='viridis')
    plt.title(f"SVM Decision Boundary with Polynomial Kernel (Degree {degree})")
    plt.xlabel('Feature 1 (standardized)')
    plt.ylabel('Feature 2 (standardized)')
    plt.show()

# Display the results summary
print("\n--- Summary of Polynomial Kernel Results ---")
for degree, metrics in results.items():
    print(f"\nDegree {degree}:")
    for metric, value in metrics.items():
        print(f"  {metric.capitalize()}: {value:.2f}")


## PARAMETER TUNING FOR POLYNOMIAL KERNEL

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Use only a sample subset for interpretation to speed up SHAP
X_train_sample, _, y_train_sample, _ = train_test_split(X_train_scaled, y_train, train_size=500, random_state=42, stratify=y_train)

# Define parameter grid for polynomial kernel tuning
param_grid = {
    'C': [0.1, 1, 10],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}

# Train the SVM model with the best parameters found for the polynomial kernel
grid_search_poly = GridSearchCV(SVC(kernel='poly', probability=True), param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search_poly.fit(X_train_sample, y_train_sample)
best_poly_model = grid_search_poly.best_estimator_

# Fit the best polynomial model to the entire training set for SHAP analysis
best_poly_model.fit(X_train_scaled, y_train)

# Make predictions on the test set (ensure you have a scaled test set)
y_pred_poly = best_poly_model.predict(X_test_scaled)  # Ensure you have defined X_test_scaled
y_proba_poly = best_poly_model.predict_proba(X_test_scaled)[:, 1]  # Probability estimates for AUC

# Calculate evaluation metrics
accuracy_poly = accuracy_score(y_test, y_pred_poly)  # Ensure you have defined y_test
precision_poly = precision_score(y_test, y_pred_poly, average='weighted')
recall_poly = recall_score(y_test, y_pred_poly, average='weighted')
f1_poly = f1_score(y_test, y_pred_poly, average='weighted')
auc_poly = roc_auc_score(y_test, y_proba_poly)  # AUC for binary classification

# Display the best model parameters and best cross-validation score
print("Best Polynomial SVM Model Parameters:")
for param, value in best_poly_model.get_params().items():
    print(f"{param}: {value}")

# Display the best cross-validation score
best_cv_score = grid_search_poly.best_score_
print(f"\nBest Cross-Validation Score (F1 Weighted): {best_cv_score:.4f}")

# Display evaluation metrics
print("\nEvaluation Metrics for Polynomial SVM:")
print(f"Accuracy: {accuracy_poly:.4f}")
print(f"Precision: {precision_poly:.4f}")
print(f"Recall: {recall_poly:.4f}")
print(f"F1 Score: {f1_poly:.4f}")
print(f"AUC: {auc_poly:.4f}")


## SVM WITH RBF KERNEL

In [ ]:


# Define and train the SVM model with an RBF kernel and default gamma='scale'
svm_model_rbf = SVC(kernel='rbf', probability=True)  # Using default gamma='scale'
svm_model_rbf.fit(X_selected, y_train)  # Assuming X_selected contains only 2 features

# Predict on the test set (using the same 2 features as X_selected)
y_pred = svm_model_rbf.predict(X_test_scaled[:, :2])  # Use the first two features for testing
y_pred_prob = svm_model_rbf.decision_function(X_test_scaled[:, :2])

# Calculate classification metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate AUC score, handling binary and multiclass cases
if len(set(y)) == 2:
    auc = roc_auc_score(y_test, y_pred_prob)
else:
    y_test_binarized = label_binarize(y_test, classes=np.unique(y))
    auc = roc_auc_score(y_test_binarized, y_pred_prob, average="weighted", multi_class="ovr")

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"AUC Score: {auc:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Plot the decision boundary for visualization
x_min, x_max = X_selected[:, 0].min() - 1, X_selected[:, 0].max() + 1
y_min, y_max = X_selected[:, 1].min() - 1, X_selected[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

Z = svm_model_rbf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
plt.scatter(X_selected[:, 0], X_selected[:, 1], c=y_train, edgecolor='k', cmap='viridis')
plt.title(f"SVM Decision Boundary with RBF Kernel (Gamma = 'scale')")
plt.xlabel('Feature 1 (standardized)')
plt.ylabel('Feature 2 (standardized)')
plt.show()


## PARAMETER TUNING FOR RBF KERNEL

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Use only a sample subset for interpretation to speed up SHAP
X_train_sample, _, y_train_sample, _ = train_test_split(X_train_scaled, y_train, train_size=500, random_state=42, stratify=y_train)

# Define parameter grid for RBF kernel tuning
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# Train the SVM model with the best parameters found for the RBF kernel
grid_search_rbf = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search_rbf.fit(X_train_sample, y_train_sample)
best_rbf_model = grid_search_rbf.best_estimator_

# Fit the best RBF model to the entire training set for SHAP analysis
best_rbf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set (ensure you have a scaled test set)
y_pred_rbf = best_rbf_model.predict(X_test_scaled) 
y_proba_rbf = best_rbf_model.predict_proba(X_test_scaled)[:, 1]  # Probability estimates for AUC

# Calculate evaluation metrics
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)  # Ensure you have defined y_test
precision_rbf = precision_score(y_test, y_pred_rbf, average='weighted')
recall_rbf = recall_score(y_test, y_pred_rbf, average='weighted')
f1_rbf = f1_score(y_test, y_pred_rbf, average='weighted')
auc_rbf = roc_auc_score(y_test, y_proba_rbf)  # AUC for binary classification

# Display the best model parameters and best cross-validation score
print("Best RBF SVM Model Parameters:")
for param, value in best_rbf_model.get_params().items():
    print(f"{param}: {value}")

# Display the best cross-validation score
best_cv_score = grid_search_rbf.best_score_
print(f"\nBest Cross-Validation Score (F1 Weighted): {best_cv_score:.4f}")

# Display evaluation metrics
print("\nEvaluation Metrics for RBF SVM:")
print(f"Accuracy: {accuracy_rbf:.4f}")
print(f"Precision: {precision_rbf:.4f}")
print(f"Recall: {recall_rbf:.4f}")
print(f"F1 Score: {f1_rbf:.4f}")
print(f"AUC: {auc_rbf:.4f}")


## SVM IMPLEMENTATION FOR SIGMOID KERNEL

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import label_binarize

# Define the Sigmoid kernel function
def sigmoid_kernel(X1, X2, alpha=1, beta=0):
    # Compute the Sigmoid kernel
    return np.tanh(alpha * np.dot(X1, X2.T) + beta)

# Create the SVM model using the custom Sigmoid kernel
svm_model_sigmoid = SVC(kernel=lambda X1, X2: sigmoid_kernel(X1, X2, alpha=1, beta=0), probability=True)

# Fit the model
svm_model_sigmoid.fit(X_train_scaled[:, :2], y_train)  # Assuming X_train_scaled contains only 2 features

# Predict on the test set
y_pred = svm_model_sigmoid.predict(X_test_scaled[:, :2])
y_pred_prob = svm_model_sigmoid.decision_function(X_test_scaled[:, :2])

# Calculate classification metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate AUC score, handling binary and multiclass cases
if len(set(y)) == 2:
    auc = roc_auc_score(y_test, y_pred_prob)
else:
    y_test_binarized = label_binarize(y_test, classes=np.unique(y))
    auc = roc_auc_score(y_test_binarized, y_pred_prob, average="weighted", multi_class="ovr")

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"AUC Score: {auc:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


## SENSITIVITY ANALYSIS FOR ALL KERNELS 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

def sensitivity_analysis_all_kernels(X, y, param_grid):
    results = []
    
    for kernel in param_grid['kernel']:
        for C in param_grid['C']:
            if kernel == 'linear':
                model = SVC(kernel=kernel, C=C)
                score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
                results.append((kernel, C, None, None, score))
                
            elif kernel == 'rbf':
                for gamma in param_grid['gamma']:
                    model = SVC(kernel=kernel, C=C, gamma=gamma)
                    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
                    results.append((kernel, C, gamma, None, score))
                    
            elif kernel == 'poly':
                for degree in param_grid['degree']:
                    model = SVC(kernel=kernel, C=C, degree=degree)
                    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
                    results.append((kernel, C, None, degree, score))
                    
            elif kernel == 'sigmoid':
                for gamma in param_grid['gamma']:
                    model = SVC(kernel=kernel, C=C, gamma=gamma)
                    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()
                    results.append((kernel, C, gamma, None, score))
    
    return pd.DataFrame(results, columns=['Kernel', 'C', 'Gamma', 'Degree', 'Accuracy'])

# Define the parameter grid for the analysis
param_grid = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': np.logspace(-2, 2, 5),  # C from 0.01 to 100
    'gamma': [0.001, 0.01, 0.1, 1],  # Different values of gamma
    'degree': [2, 3, 4]  # Degrees for polynomial
}

# Run the sensitivity analysis
sensitivity_results = sensitivity_analysis_all_kernels(X, y, param_grid)
print(sensitivity_results)


## PLOTTING HEATMAPS

In [ ]:
def plot_heatmaps(sensitivity_results):
    kernels = sensitivity_results['Kernel'].unique()
    
    for kernel in kernels:
        kernel_data = sensitivity_results[sensitivity_results['Kernel'] == kernel]
        
        if kernel == 'linear':
            heatmap_data = kernel_data.pivot_table(index='C', values='Accuracy')
            plt.figure(figsize=(6, 5))
            sns.heatmap(heatmap_data, annot=True, cmap='viridis', cbar_kws={'label': 'Accuracy Score'})
            plt.title(f'Sensitivity Analysis Heatmap (SVM with {kernel.capitalize()} Kernel)')
            plt.xlabel('C')
            plt.ylabel('Accuracy')
            plt.show()
        
        elif kernel == 'rbf':
            heatmap_data = kernel_data.pivot_table(index='C', columns='Gamma', values='Accuracy')
            plt.figure(figsize=(8, 6))
            sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt=".2f", cbar_kws={'label': 'Accuracy Score'})
            plt.title(f'Sensitivity Analysis Heatmap (SVM with {kernel.capitalize()} Kernel)')
            plt.xlabel('Gamma')
            plt.ylabel('C')
            plt.show()
        
        elif kernel == 'poly':
            heatmap_data = kernel_data.pivot_table(index='C', columns='Degree', values='Accuracy')
            plt.figure(figsize=(8, 6))
            sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt=".2f", cbar_kws={'label': 'Accuracy Score'})
            plt.title(f'Sensitivity Analysis Heatmap (SVM with {kernel.capitalize()} Kernel)')
            plt.xlabel('Degree')
            plt.ylabel('C')
            plt.show()
        
        elif kernel == 'sigmoid':
            heatmap_data = kernel_data.pivot_table(index='C', columns='Gamma', values='Accuracy')
            plt.figure(figsize=(8, 6))
            sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt=".2f", cbar_kws={'label': 'Accuracy Score'})
            plt.title(f'Sensitivity Analysis Heatmap (SVM with {kernel.capitalize()} Kernel)')
            plt.xlabel('Gamma')
            plt.ylabel('C')
            plt.show()

# Plot the heatmaps for the results
plot_heatmaps(sensitivity_results)


In [ ]:
def plot_degree_sensitivity(sensitivity_results, specific_C):
    degree_results = sensitivity_results[(sensitivity_results['C'] == specific_C) & 
                                         (sensitivity_results['Kernel'] == 'poly')]
    
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=degree_results, x='Degree', y='Accuracy', marker='o')  # Changed 'ROC AUC' to 'Accuracy'
    plt.title(f'SVM Performance Sensitivity to Kernel Degree (C={specific_C})')
    plt.xlabel('Degree of Polynomial Kernel')
    plt.ylabel('Accuracy Score')  # Changed label to 'Accuracy Score'
    plt.xticks(degree_results['Degree'])
    plt.grid()
    plt.show()

# Example usage for degree sensitivity visualization
plot_degree_sensitivity(sensitivity_results, specific_C=10)  # Adjust C as needed


## SHAP FOR FEATURE CONTRIBUTIONS IN THE OUTPUT

In [ ]:
import numpy as np
import shap
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Assume X_train and X_test are your training and test features and y_train and y_test are the labels

# Scale your features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the Sigmoid kernel function
def sigmoid_kernel(X1, X2, alpha=1, beta=0):
    return np.tanh(alpha * np.dot(X1, X2.T) + beta)

# Create and fit the SVM model
svm_model_sigmoid = SVC(kernel=lambda X1, X2: sigmoid_kernel(X1, X2, alpha=1, beta=0), probability=True)
svm_model_sigmoid.fit(X_train_scaled, y_train)

# Predictions
y_pred = svm_model_sigmoid.predict(X_test_scaled)

# Print accuracy and classification report
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Step 3: Use SHAP to explain predictions
# Randomly sample a subset of the test set for SHAP
sample_size = 100  # Set the sample size
sample_indices = np.random.choice(X_test_scaled.shape[0], size=sample_size, replace=False)
X_sample = X_test_scaled[sample_indices]

explainer = shap.KernelExplainer(svm_model_sigmoid.predict_proba, X_train_scaled)
shap_values = explainer.shap_values(X_sample)

# Step 4: Plot SHAP summary for the sampled data
shap.summary_plot(shap_values, X_sample, feature_names=[f'Feature {i}' for i in range(X_sample.shape[1])])
